# <br/><br/><br/><br/><br/>Esta bitacora procesa datos obtenidos de un proteoma<br/><br/><br/><br/><br/>

# Paquetes<br/>

In [ ]:
## Paquetes
library(colorRamps)
library(cowplot)
library(dplyr)
library(ggplot2)
library(ggpubr)
#library(GOplot)
library(grid)
library(gridExtra)
library(IRdisplay)
library(plyr)
library(RColorBrewer)
library(reshape2)
library(RSVGTipsDevice)
library(scales)
library(shiny)
library(shinythemes)
library(UpSetR)
library(RCurl)
library(wesanderson)
library(VennDiagram)
library(tidyr)

## Arreglos para los graficos<br/>

In [ ]:
## configuracion del fondo y bordes para los graficos pie chart y bar plot
pie_blank_theme = theme_minimal()+
  theme(
  axis.text.x=element_blank(),
  axis.title.x = element_blank(),
  axis.title.y = element_blank(),
  panel.border = element_blank(),
  panel.grid=element_blank(),
  axis.ticks = element_blank(),
  )
bar_blank_theme = theme(
axis.ticks.y=element_blank(),
panel.background = element_rect(fill = "white"),#cambia el color del fondo del grafico
axis.ticks = element_blank(), # elimina las marcas en la linea de escala de los ejes x y
axis.text.x = element_blank(),#elimina la escala del eje x
axis.line=element_blank(),
axis.ticks.x=element_blank(),
)

In [ ]:
colors=c(brewer.pal(12,"Set3"),brewer.pal(8,"Set2"),brewer.pal(12,"Paired"),brewer.pal(8,"Dark2"),
         brewer.pal(8,"Accent"),brewer.pal(12,"Set3"),brewer.pal(8,"Set2"),brewer.pal(12,"Paired"),
         brewer.pal(8,"Dark2"),brewer.pal(8,"Accent"),brewer.pal(12,"Set3"),brewer.pal(8,"Set2"),
         brewer.pal(12,"Paired"),brewer.pal(8,"Dark2"),brewer.pal(8,"Accent"))

In [ ]:
dir.create('~/Desktop/data/Experimento1_GOslim/images') ## creamos un directorio donde se depositaran los graficos creados

# Plot 1: Pie chart Total Proteins/GO terms for C F P<br/>

In [ ]:
frec_CFP=read.csv("../data/Experimento1_GOslim/Total_categories_CFP.csv",header =F)
frec_CFP

In [ ]:
colnames(frec_CFP)=c("Aspect","Value") ## para cambiar la etiqueta del data frame 
#total_CFP=sum(frec_CFP$Value)
#total_CFP
frec_CFP

In [ ]:
total_CFP = frec_CFP %>% 
mutate(Aspect=factor(Aspect,levels=Aspect[length(Aspect):1]),
      cumulative=cumsum(Value),
      midpoint=cumulative - (Value/2),
      #labels=paste0(round((Value/sum(Value))*100,1),"%"))
      #labels=paste0(round((Value/sum(Value))*100,1),"%"," (",Value,") "))
      labels=paste0(round((Value/sum(Value))*100,1),"%"))
#table_labels  ## activar para visualizar 
total_CFP_secretoma=ggplot(total_CFP, aes(x = "", y = Value,fill =Aspect))+
pie_blank_theme+
geom_bar(width = 1,stat = "identity")+
        coord_polar(theta = "y",start = 0)+
scale_fill_manual(values = colors)+
        #scale_fill_brewer(palette = "Paired")+
        #scale_fill_manual(values = color) +
        labs(x = "",y = "",title = "Total GO terms",fill="Aspect")+
geom_text(aes(x = 1.1, y = midpoint,label = labels),color="black",
          fontface = "bold",size = 5)+
theme(plot.title = element_text(hjust =0.5,,size=15,margin=margin(b =-10,unit="pt"),face="bold"),
legend.text=element_text(colour="black",size=12),#color y tamaño del texto de la leyenda
legend.margin=margin(b = c(0,-1), unit = "cm"),#distancia entre la leyenda y el grafico
legend.title = element_text(hjust = 0.5,face="bold",size = 12))
ggsave("../data/Experimento1_GOslim/images/Total_categories_CFP.png",width=7,height=7,dpi=600)
total_CFP_secretoma

# Intersecciones

# Plot 2: Venn Diagram for F C P<br/>

In [ ]:
cats=read.csv("../data/Experimento1_GOslim/cats.csv",header=TRUE)
head(cats)

In [ ]:
procc=dplyr::filter(cats,  !is.na(Process))
proceso=as.character(procc$Process)
funcc=dplyr::filter(cats,  !is.na(Function))
funcion=as.character(funcc$Function)
compp=dplyr::filter(cats,  !is.na(Component))
componente=as.character(compp$Component)

In [ ]:
## Una forma de visualizar las agrupaciones
overlapping = calculate.overlap(x= list("funcion" = funcion,"componente" = componente,"proceso"=proceso));
overlapping   ##<<<<< activar la orden para ver la distribucion de los conjuntos

In [ ]:
## solo visualizacion del diagrama de Venn en alta resolucion
plot=venn.diagram(list("F" = funcion,"C" = componente,"P"=proceso),
              fill = colors[c(24,28,30)], #c('yellow', 'purple', 'green'),
              main = "Categories F C P", main.cex = 1.7,main.fontface=2,
              alpha = c(0.4,0.4,0.4), cex = 1.5,cat.fontface = 1,cat.fontfamily = "sans",
              lty =0, filename=NULL, cat.cex=2, cat.dist = c(0.045, 0.02, 0.04),
              cat.just=list(c(0.4,1) , c(0,0) , c(0,0)))
grid.newpage()
grid.draw(plot)
## por cada ejecucion de este comando se genera un historial en el directorio,
## los archivos tienen extencioń .log, se pueden eliminar para no saturar el directorio.

In [ ]:
## si esta satisfecho con el diagrama anterior, salvar la imagen con la siguiente orden
venn.diagram(list("F" = funcion,"C" = componente,"P"=proceso),
              fill =  colors[c(24,28,30)],#c('yellow', 'purple', 'green'),main = "Categories F C P", main.cex = 2,
              alpha = c(0.3, 0.3,0.3), cex = 2.5,cat.fontface = 2,cat.fontfamily = "sans",
              lty =0, filename='../data/Experimento1_GOslim/images/frecuencias_F_C_P_venn.tiff', cat.cex=2, cat.dist = c(0.045, 0.02, 0.05),
              cat.just=list(c(0.6,1) , c(0,0) , c(0,0)))
## Con este comando eliminamos los reportes que se generan cada vez que visualizamos o salvamos este grafico,  
delete_report=dir(path="~/Desktop/data/Experimento1_GOslim/images/",pattern="[0-9].log",recursive=T,full.names=T) ## se guarda una lista de ficheros con extension .log
delete_report2=dir(path="~/Desktop/jupyter/",pattern="[0-9].log",recursive=T,full.names=T) ## se guarda una lista de ficheros con extension .log
file.remove(delete_report) ## elimina todos los ficheros con extension .log
file.remove(delete_report2) ## elimina todos los ficheros con extension .log

# Plot 3: Pie chart / Bar / Intersections plot of Molecular Function
## Pie Chart

In [ ]:
## abrimos el fichero los resultados para proceso
aspect_f=read.csv("../data/Experimento1_GOslim/Molecular_Function.csv")
head(aspect_f)

    * mutate: permite agregar una nueva columna o transformar una existente
    * pipe (%>%) es un operador que permite encadenar funciones, esto ayuda enormemente a mejorar la legibilidad del código

In [ ]:
## 
f_proportion= aspect_f%>% #preparamos la variable para aplicarle funciones
mutate(f.prop=Value/sum(Value))%>%
mutate(f.proportion=percent(Value/sum(Value)))%>% #obtenemos el "%" y lo agregamos al df inicial
mutate(Function=paste(Molecular.Function,f.proportion))%>% #fusionamos los terminos y el "%" para visualizacion
#convertimos los datos de la columna Process a una variable categorica (factor)
#porque inicialmente es reconocida como caracter y no permite ordenar los datos en el grafico
#con la siguiente orden condiguramos el orden de los datos en el grafico y la leyenda
mutate(Function=factor(Function,levels=Function[1:length(Function)]))
#[length(Process):0] con esta orden se cambia el sentido de los datos en el grafico
# con "class" podemos ver la clase del objeto
head(f_proportion)

In [ ]:
pie_molecular_function=ggplot(f_proportion,aes(x="",y=Value,fill=Function))+#introduccion de datos a graficar
pie_blank_theme+#elimina el color de fondo del grafico
geom_bar(width =1,stat ="identity")+#
coord_polar(theta="y",start=0)+#orientacion del grafico, empieza en 0°
scale_fill_manual(values=colors)+# colores del grafico
ggtitle("Molecular Function") + # agrega titulo al grafico
theme(legend.title=element_text(colour="white",size=7,face="bold"),#color, tamaño y formato del titulo de la leyenda
legend.text=element_text(colour="black",size=8),#color y tamaño del texto de la leyenda
plot.title=element_text(hjust=.5,size=12,margin=margin(b =-12,unit="pt"),face="bold"),# posicion,tamaño del titulo, y espacio entre el titulo y el grafico
legend.key.width=unit(0.2,"cm"),#ancho de la figura de la leyenda
legend.key.height=unit(0.4,"cm"),#alto de la figura de la leyenda, espacio entre el texto de la leyenda
legend.margin=margin(b = c(.1,-1), unit = "cm"),#distancia entre la leyenda y el grafico
legend.position= "right", #"bottom",
plot.margin=margin(.5,2,5,.1,"cm"))#margenes exteriores de todo el grafico
ggsave("../data/Experimento1_GOslim/images/Pie_Molecular_Function.png",width=7,height=7,dpi=600)#,compression='lzw')#guarda y reedita la figura guardada cada vez que se hacen cambios y se ejecuta el comando
pie_molecular_function
#display_png(file="/home/lalo/Escritorio/curso/plots/pie_biological_process.png")

## Bar Plot

In [ ]:
bar_molecular_function=ggplot(f_proportion,aes(x=Molecular.Function,y=f.prop))+#introduccion de datos a graficar
geom_bar(width=0.6,fill=rev(colors[1:nrow(aspect_f)]),stat ="identity")+#ancho y color de relleno de las barras
labs(x ="",y ="",title="Molecular Function")+#titulo de eje y, x y titulo de grafico
coord_flip()+#para crear graficos horizontales
bar_blank_theme+#elimina el color de fondo del grafico
#scale_x_discrete(limits=p_proportion$Biological.Process)+  #orden descendente del grafico
scale_x_discrete(limits=rev(f_proportion$Molecular.Function))+ #orden ascendente del grafico
geom_text(aes(x=Molecular.Function,y=f.prop+0.012,#configura el espacio entre la etiqueta de datos y el eje x  
label=percent(f.prop)),size=2.5)+#configura el valor de la etiqueta de datos y el tamaño de letra
theme(plot.title = element_text(hjust = .1,size = 12,face="bold"),#posocion y tamaño del titulo sobre el grafico
axis.text.y = element_text(size = 8,color="black"),#tamaño y color de texto de eje x
axis.ticks.length=unit(-.3, "cm"), #reduce la longitud de las marcas en la linea de escala de los ejes x y
plot.margin = margin(1,.5,9,.1, "cm"))#margenes exteriores del grafico
ggsave("../data/Experimento1_GOslim/images/Bar_Molecular_Function.png",width=7,height=7,dpi=600)#,compression='lzw')
#ggsave("./plots/bar_biological_process.pdf",width=7,height=7,dpi=600) #pdf
bar_molecular_function
#display_png(file="/home/lalo/Escritorio/curso/plots/bar_biological_process.png")

# Other intersection form

In [ ]:
display_jpeg(RCurl::getBinaryURL('https://raw.githubusercontent.com/eduardo1011/Programas/master/interactios.jpg'))

## Intersections

In [ ]:
head(aspect_f)

In [ ]:
# check.names=FALSE  hace que no se pierdan los dos puntos en los names de las columnas
up_function=read.csv("../data/Experimento1_GOslim/Molecular_Function_matrix.csv",header=TRUE,check.names=FALSE)
head(up_function)

In [ ]:
## Seleccion de GO IDs de interes para ver sus intersecciones
go_function=as.character(aspect_f$GO[1:6])
go_function

In [ ]:
## intersecciones entre terminos, esto puede sustituir diagramas de ven con mas de 5 conjuntos
##
## c(intersection size title, intersection size tick labels, set size title, set size tick labels, set names, numbers above bars)
#      text.scale = c(1.5,1.5,0,1.5,1,1.5),
##
png(file="../data/Experimento1_GOslim/images/Function_Intersections.png",width = 10,height = 10,units = "in",res=500,bg="white")
up_fun= function() {
    upset(up_function,sets=go_function,
          sets.bar.color = colors[1:length(go_function)],order.by = "freq",empty.intersections = NULL,point.size=2,
          mainbar.y.label="Functional annotation by GO (MF)",main.bar.color=colors[10],
          matrix.color="black",shade.color=colors[6],line.size=0.5,show.numbers = "yes",group.by = "degree",
          text.scale = c(1.5,1.5,0,1.5,1,1.5),
          matrix.dot.alpha = 0.5,mb.ratio = c(0.7, 0.3))
}
up_fun()
dev.off()
#
up_fun()

# Plot 4: Pie chart / Bar / Intersections plot of Biological process
## Pie Chart

In [ ]:
## abrimos el fichero los resultados para proceso
aspect_p=read.csv("../data/Experimento1_GOslim/Biological_Process.csv")
head(aspect_p)

    * mutate: permite agregar una nueva columna o transformar una existente
    * pipe (%>%) es un operador que permite encadenar funciones, esto ayuda enormemente a mejorar la legibilidad del código

In [ ]:
##  
p_proportion= aspect_p%>% #preparamos la variable para aplicarle funciones
mutate(p.prop=Value/sum(Value))%>%
mutate(p.proportion=percent(Value/sum(Value)))%>% #obtenemos el "%" y lo agregamos al df inicial
mutate(Process=paste(Biological.Process,p.proportion))%>% #fusionamos los terminos y el "%" para visualizacion
#convertimos los datos de la columna Process a una variable categorica (factor)
#porque inicialmente es reconocida como caracter y no permite ordenar los datos en el grafico
#con la siguiente orden condiguramos el orden de los datos en el grafico y la leyenda
mutate(Process=factor(Process,levels=Process[1:length(Process)]))
#[length(Process):0] con esta orden se cambia el sentido de los datos en el grafico
# con "class" podemos ver la clase del objeto
head(p_proportion)

In [ ]:
pie_biological_process=ggplot(p_proportion,aes(x="",y=Value,fill=Process))+#introduccion de datos a graficar
pie_blank_theme+#elimina el color de fondo del grafico
geom_bar(width =1,stat ="identity")+#
coord_polar(theta="y",start=0)+#orientacion del grafico, empieza en 0°
scale_fill_manual(values=colors)+# colores del grafico
ggtitle("Biological Process") + # agrega titulo al grafico
theme(legend.title=element_text(colour="white",size=7,face="bold"),#color, tamaño y formato del titulo de la leyenda
legend.text=element_text(colour="black",size=8),#color y tamaño del texto de la leyenda
plot.title=element_text(hjust=.5,size=12,margin=margin(b =-12,unit="pt"),face="bold"),# posicion,tamaño del titulo, y espacio entre el titulo y el grafico
legend.key.width=unit(0.2,"cm"),#ancho de la figura de la leyenda
legend.key.height=unit(0.4,"cm"),#alto de la figura de la leyenda
legend.margin=margin(b = c(.1,-1), unit = "cm"),#distancia entre la leyenda y el grafico
legend.position= "right", #"bottom",
plot.margin=margin(.5,1,5,.1,"cm"))#margenes exteriores de todo el grafico
ggsave("../data/Experimento1_GOslim/images/Pie_Biological_Process.png",width=7,height=7,dpi=600)#,compression='lzw')#guarda y reedita la figura guardada cada vez que se hacen cambios y se ejecuta el comando
pie_biological_process
#display_png(file="/home/lalo/Escritorio/curso/plots/pie_biological_process.png")

## Bar Plot

In [ ]:
bar_biological_process=ggplot(p_proportion,aes(x=Biological.Process,y=p.prop))+#introduccion de datos a graficar
geom_bar(width=0.6,fill=rev(colors[1:nrow(aspect_p)]),stat ="identity")+#ancho y color de relleno de las barras
labs(x ="",y ="",title="Biological process")+#titulo de eje y, x y titulo de grafico
coord_flip()+#para crear graficos horizontales
bar_blank_theme+#elimina el color de fondo del grafico
#scale_x_discrete(limits=p_proportion$Biological.Process)+  #orden descendente del grafico
scale_x_discrete(limits=rev(p_proportion$Biological.Process))+ #orden ascendente del grafico
geom_text(aes(x=Biological.Process,y=p.prop+0.012,#configura el espacio entre la etiqueta de datos y el eje x  
label=percent(p.prop)),size=2.5)+#configura el valor de la etiqueta de datos y el tamaño de letra
theme(plot.title = element_text(hjust = .1,size = 12,face="bold"),#posocion y tamaño del titulo sobre el grafico
axis.text.y = element_text(size = 8,color="black"),#tamaño y color de texto de eje x
axis.ticks.length=unit(-.3, "cm"), #reduce la longitud de las marcas en la linea de escala de los ejes x y
plot.margin = margin(1,.5,9,.1, "cm"))#margenes exteriores del grafico
ggsave("../data/Experimento1_GOslim/images/Bar_Biological_Process.png",width=7,height=7,dpi=600)#,compression='lzw')
#ggsave("./plots/bar_biological_process.pdf",width=7,height=7,dpi=600) #pdf
bar_biological_process
#display_png(file="/home/lalo/Escritorio/curso/plots/bar_biological_process.png")

## Intersections

In [ ]:
head(aspect_p)

In [ ]:
# check.names=FALSE  hace que no se pierdan los dos puntos en los names de las columnas
up_process=read.csv("../data/Experimento1_GOslim/Biological_Process_matrix.csv",header=TRUE,check.names=FALSE)
head(up_process)

In [ ]:
## Seleccion de GO IDs de interes para ver sus intersecciones
go_process=as.character(aspect_p$GO[1:6])
go_process

In [ ]:
## intersecciones entre términos, esto puede sustituir diagramas de ven con más de 5 conjuntos
png(file="../data/Experimento1_GOslim/images/Process_Intersections.png",width = 10,height = 10,units = "in",res=500,bg="white")
up_proc= function() {
    upset(up_process,sets=go_process,
          sets.bar.color = colors[1:6],order.by = "freq",empty.intersections = NULL,point.size=2,
          mainbar.y.label="Functional annotation by GO (BP)",main.bar.color=colors[10],
          matrix.color="black",shade.color=colors[6],line.size=0.5,show.numbers = "yes",group.by = "degree",
          text.scale = c(1.5,1.5,0,1.5,1,1.5),
          matrix.dot.alpha = 0.5,mb.ratio = c(0.7, 0.3))
}
up_proc()
dev.off()
#
up_proc()

# Plot 5: Pie chart / Bar / Intersections plot of Cellular Component
## Pie Chart

In [ ]:
## abrimos el fichero los resultados para proceso
aspect_c=read.csv("../data/Experimento1_GOslim/Cellular_Component.csv")
head(aspect_c)

    * mutate: permite agregar una nueva columna o transformar una existente
    * pipe (%>%) es un operador que permite encadenar funciones, esto ayuda enormemente a mejorar la legibilidad del código

In [ ]:
## solo estoy egraficando los primeros 15 terminos 
c_proportion= aspect_c%>% #preparamos la variable para aplicarle funciones
mutate(c.prop=Value/sum(Value))%>%
mutate(c.proportion=percent(Value/sum(Value)))%>% #obtenemos el "%" y lo agregamos al df inicial
mutate(Component=paste(Cellular.Component,c.proportion))%>% #fusionamos los terminos y el "%" para visualizacion
#convertimos los datos de la columna Process a una variable categorica (factor)
#porque inicialmente es reconocida como caracter y no permite ordenar los datos en el grafico
#con la siguiente orden condiguramos el orden de los datos en el grafico y la leyenda
mutate(Component=factor(Component,levels=Component[1:length(Component)]))
#[length(Process):0] con esta orden se cambia el sentido de los datos en el grafico
# con "class" podemos ver la clase del objeto
head(c_proportion)

In [ ]:
pie_cellular_component=ggplot(c_proportion,aes(x="",y=Value,fill=Component))+#introduccion de datos a graficar
pie_blank_theme+#elimina el color de fondo del grafico
geom_bar(width =1,stat ="identity")+#
coord_polar(theta="y",start=0)+#orientacion del grafico, empieza en 0°
scale_fill_manual(values=colors)+# colores del grafico
ggtitle("Cellular Component") + # agrega titulo al grafico
theme(legend.title=element_text(colour="white",size=7,face="bold"),#color, tamaño y formato del titulo de la leyenda
legend.text=element_text(colour="black",size=8),#color y tamaño del texto de la leyenda
plot.title=element_text(hjust=.5,size=12,margin=margin(b =-12,unit="pt"),face="bold"),# posicion,tamaño del titulo, y espacio entre el titulo y el grafico
legend.key.width=unit(0.2,"cm"),#ancho de la figura de la leyenda
legend.key.height=unit(0.4,"cm"),#alto de la figura de la leyenda
legend.margin=margin(b = c(.1,-1), unit = "cm"),#distancia entre la leyenda y el grafico
legend.position= "right", #"bottom",
plot.margin=margin(.5,4,5,.1,"cm"))#margenes exteriores de todo el grafico
ggsave("../data/Experimento1_GOslim/images/Pie_Cellular_Component.png",width=7,height=7,dpi=600)#,compression='lzw')#guarda y reedita la figura guardada cada vez que se hacen cambios y se ejecuta el comando
pie_cellular_component
#display_png(file="/home/lalo/Escritorio/curso/plots/pie_biological_process.png")

## Bar

In [ ]:
bar_cellular_component=ggplot(c_proportion,aes(x=Cellular.Component,y=c.prop))+#introduccion de datos a graficar
geom_bar(width=0.6,fill=rev(colors[1:nrow(aspect_c)]),stat ="identity")+#ancho y color de relleno de las barras
labs(x ="",y ="",title="Cellular Component")+#titulo de eje y, x y titulo de grafico
coord_flip()+#para crear graficos horizontales
bar_blank_theme+#elimina el color de fondo del grafico
#scale_x_discrete(limits=p_proportion$Biological.Process)+  #orden descendente del grafico
scale_x_discrete(limits=rev(c_proportion$Cellular.Component))+ #orden ascendente del grafico
geom_text(aes(x=Cellular.Component,y=c.prop+0.01,#configura el espacio entre la etiqueta de datos y el eje x  
label=percent(c.prop)),size=2.5)+#configura el valor de la etiqueta de datos y el tamaño de letra
theme(plot.title = element_text(hjust = .1,size = 12,face="bold"),#posocion y tamaño del titulo sobre el grafico
axis.text.y = element_text(size = 8,color="black"),#tamaño y color de texto de eje x
axis.ticks.length=unit(-.5, "cm"), #reduce la longitud de las marcas en la linea de escala de los ejes x y
plot.margin = margin(1,.5,9,.1, "cm"))#margenes exteriores del grafico
ggsave("../data/Experimento1_GOslim/images/Bar_Cellular_Component.png",width=7,height=7,dpi=600)#,compression='lzw')
#ggsave("./plots/bar_biological_process.pdf",width=7,height=7,dpi=600) #pdf
bar_cellular_component
#display_png(file="/home/lalo/Escritorio/curso/plots/bar_biological_process.png")

## Intersections

In [ ]:
head(aspect_c)

In [ ]:
# check.names=FALSE  hace que no se pierdan los dos puntos en los names de las columnas
up_component=read.csv("../data/Experimento1_GOslim/Cellular_Component_matrix.csv",header=TRUE,check.names=FALSE)
head(up_component)

In [ ]:
## Seleccion de GO IDs de interes para ver sus intersecciones
go_component=as.character(aspect_c$GO[1:6])
go_component

In [ ]:
## intersecciones entre términos, esto puede sustituir diagramas de ven con más de 5 conjuntos
png(file="../data/Experimento1_GOslim/images/Component_Intersections.png",width = 10,height = 10,units = "in",res=500,bg="white")
up_comp=function(){
    upset(up_component,sets=go_component,
          sets.bar.color = colors[1:6],order.by = "freq",empty.intersections = NULL,point.size=2,
          mainbar.y.label="Functional annotation by GO (CC)",main.bar.color=colors[10],
          matrix.color="black",shade.color=colors[6],line.size=0.5,show.numbers = "yes",group.by = "degree",
          text.scale = c(1.5,1.5,0,1.5,1,1.5),
          matrix.dot.alpha = 0.5,mb.ratio = c(0.7, 0.3))
}
up_comp()
dev.off()
#
up_comp()